In [1]:
import pickle

import os 
import glob
import random
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import cv2
import numpy as np
import os.path as osp
import sys

ws_dir = '/local/home/ekoller/BT'
print(ws_dir)
sys.path.append(ws_dir)
from utils import scan3r

data_dir ='/local/home/ekoller/R3Scan'
scenes_dir = '/local/home/ekoller/R3Scan/scenes'
#scan_id= "38770c95-86d7-27b8-8717-3485b411ddc7" #is reference scan  since it is a reference scan everything shouls be correctly hit
curr_scan_id = "02b33e01-be2b-2d54-93fb-4145a709cec5" 
new_scan_id =  "fcf66d8a-622d-291c-8429-0e1109c6bb26"
frame_number = "000182"

/local/home/ekoller/BT


this codesegment does the following: given an image pose: find the closes pose: method if the pose is given in a rescan coordinate turn it into a reference coordinate for the comparison

In [2]:
def pose_in_reference(data_dir,scan_id, pose_rescan):
    #for rescan no need to change that
    if scan3r.is_rescan(data_dir,scan_id):
        return pose_rescan

    #get the path to the matricies of each scan_id for transformation of rescan to reference
    path = osp.join(data_dir,"files", "3RScan.json")
    #access the particular rescan_scan_id
    ref2_rescan_all_id = scan3r.read_transform_mat(path)
    ref2rescan = ref2_rescan_all_id[scan_id] 
    rescan2ref = np.linalg.inv(ref2rescan)

    #based on rayintersection where we transformed the extrinsic matrix we now just transforme the pose
    pose_reference = pose_rescan * rescan2ref.transpose()

    return pose_reference
    

pose_in_reference(data_dir,new_scan_id, frame_number)

'000182'

In [10]:
def rotational_difference(rot_1, rot_2):
    # Rotation difference = acos((trace(R1^T * R2)-1)/2)
    R = np.dot(rot_1.T, rot_2)  
    trace_R = np.trace(R)
    angle = np.arccos((trace_R - 1) / 2)
    return angle


In [67]:
#given a frame with a pose of a new scan id, find the closes pose in the current scan id
def closest_pose(data_dir, curr_scan_id, new_scan_id, frame_number):
    #get the pose of the frame
    pose_rescan = scan3r.load_pose(osp.join(data_dir, "scenes"), new_scan_id, frame_number)
    print("pose rescan shape", pose_rescan.shape)

    #transfrom it if necessary
    pose_reference = pose_in_reference(data_dir,new_scan_id, pose_rescan)
    print("pose referemce", pose_reference)

    #get all the poses of the current scan
    curr_poses =scan3r.load_frame_poses_paths(data_dir, curr_scan_id)

    #extrac the pose reference rotation and translation
    ref_rot = pose_reference[:3,:3]

    #get the camera position
    ref_cam = scan3r.find_cam_centers([frame_number], [pose_reference])
    

    ref_trans = pose_reference[:3,3] 


    #initialize
    min_distance = float('inf')
    closest_matrix = None
    closest_frame = None


    #go through each of our poses
    for frame, pose_path in curr_poses.items():
        pose = np.loadtxt(pose_path)
        print("pose", pose)
        frame_rot = pose[:3,:3]
        print("frame_ rot", frame_rot)
        #frame_trans = pose[:3,3]
        frame_trans = scan3r.find_cam_centers([frame], [pose])
        print("frame trans", frame_trans)

        rot_diff = rotational_difference(ref_rot, frame_rot)
        trans_diff = np.linalg.norm(ref_cam - frame_trans)

        combined_metric = rot_diff + trans_diff

        #update if necessary
        if combined_metric < min_distance:
            min_distance = combined_metric
            closest_matrix = pose
            closest_frame = frame

    return closest_frame, min_distance,closest_matrix




In [68]:
frame,min_distance, pose =closest_pose(data_dir, curr_scan_id, new_scan_id, frame_number)

print("closes frame", frame, "error", min_distance)

pose rescan shape (4, 4)
pose referemce [[ 0.458862  -0.711828  -0.531738  -0.68592  ]
 [-0.428059  -0.701539   0.569744   0.566052 ]
 [-0.778595  -0.0338187 -0.626615   0.0379922]
 [ 0.         0.         0.         1.       ]]
pose [[-0.0287759  -0.947845    0.31743     0.114951  ]
 [ 0.00221559  0.317501    0.948256   -0.0669439 ]
 [-0.999583    0.0279901  -0.00703633  0.142472  ]
 [ 0.          0.          0.          1.        ]]
frame_ rot [[-0.0287759  -0.947845    0.31743   ]
 [ 0.00221559  0.317501    0.948256  ]
 [-0.999583    0.0279901  -0.00703633]]
frame trans [[ 0.11495095 -0.06694375  0.14247211]]
pose [[ 0.00373418 -0.98635     0.164619    0.0811253 ]
 [ 0.154021    0.163223    0.974493   -0.0671852 ]
 [-0.988061    0.0217158   0.152528    0.192406  ]
 [ 0.          0.          0.          1.        ]]
frame_ rot [[ 0.00373418 -0.98635     0.164619  ]
 [ 0.154021    0.163223    0.974493  ]
 [-0.988061    0.0217158   0.152528  ]]
frame trans [[ 0.08112533 -0.06718513  0.